In [ ]:
def filter_entities(entities, filter_dict):
    op_map = {
        'equals': lambda x, y: x == y,
        'greater_than': lambda x, y: x > y,
        'less_than': lambda x, y: x < y,
        'contains': lambda x, y: y in x,
        'matches': lambda x, y: re.search(y, x)
    }

    def match(entity, rule):
        prop = rule['property']
        condition = rule.get('condition', 'and')
        value = rule['value']
        return op_map[condition](entity.get(prop), value)

    def evaluate(entity, filter_group):
        if 'property' in filter_group:
            return match(entity, filter_group)

        results = [evaluate(entity, rule) for rule in filter_group['rules']]
        return all(results) if filter_group.get("condition", "and") == 'and' else any(results)

    return [entity for entity in entities if evaluate(entity, filter_dict)]

# Example usage
entities = [
    {'author': 'J.K. Rowling', 'publishedYear': 2001, 'genre': 'Fantasy', 'rating': 4},
    {'author': 'George R.R. Martin', 'publishedYear': 1996, 'genre': 'Fantasy', 'rating': 5},
    {'author': 'J.R.R. Tolkien', 'publishedYear': 1954, 'genre': 'Fantasy', 'rating': 5},
    {'author': 'Isaac Asimov', 'publishedYear': 1951, 'genre': 'Science Fiction', 'rating': 4}
]

filter_dict = {
    'entityType': 'Book',
    'condition': 'or',
    'rules': [
        {
            'condition': 'and',
            'rules': [
                {'property': 'author', 'condition': 'equals', 'value': 'J.K. Rowling'},
                {'property': 'publishedYear', 'condition': 'greater_than', 'value': 2000}
            ]
        },
        {
            'condition': 'and',
            'rules': [
                {'property': 'genre', 'condition': 'contains', 'value': 'Fantasy'},
                {'property': 'rating', 'condition': 'equals', 'value': 5}
            ]
        }
    ]
}

filter_dict = {
    'entityType': 'Book',
    'rules': [
        {'property': 'author', 'condition': 'equals', 'value': 'J.K. Rowling'},
    ]
}

filtered = filter_entities(entities, filter_dict)
print(filtered)
import yaml

filter_dict = {
    'entityType': 'Book',
    'rules': [
        {'property': 'tags', 'condition': 'contains', 'value': 'tagname'},
    ]
}

print(yaml.dump(filter_dict))

In [ ]:
import glob

# find all md files in docs including subdirectories
tools = glob.glob('docs/**/*.md', recursive=True)

tools

In [12]:
import re
import yaml

for tool in tools:
    with open(tool, 'r') as f:
        content = f.read()
    # check if contains [usecase=lehrende] or [risks] or similar tags with regex
    if re.search(r'\[[a-z0-9,=\-]+\]', content):
        print(tool)
        result = re.findall(r'\[[a-z0-9,=\- ]+\]', content)
        # print all matches from regex
        for r in result:
            filter_dict = {
                'entityType': 'Book',
                'rules': [
                    {'property': 'tags', 'condition': 'contains', 'value': 'tagname'},
                ]
            }
            entity = r[1:-1].split('=')[0]
            if r.find('=') == -1:
                filters = []
            else:
                filters = r[1:-1].split('=')[1].split(',')

            filter_dict = {
                'entityType': entity,
                'condition': 'or',
                'rules': [
                    {'property': 'id', 'condition': 'contains', 'value': f} for f in filters
                ] if filters else []
            }
            print(r)
            print(yaml.dump(filter_dict))
            # replace all with yaml
            content = content.replace(r, "```yaml\n"+yaml.dump(filter_dict)+"```\n")

        # write back to file
        with open(tool, 'w') as f:
            f.write(content)





docs/anwendungsfälle/textuebersetzen.md
[chance=zuganglichkeit-247]
condition: or
entityType: chance
rules:
- condition: contains
  property: id
  value: zuganglichkeit-247

docs/anwendungsfälle/unterlagenubersetzung.md
[chance=zuganglichkeit-247]
condition: or
entityType: chance
rules:
- condition: contains
  property: id
  value: zuganglichkeit-247

docs/anwendungsfälle/inhaltverbessern.md
[chance=zuganglichkeit-247]
condition: or
entityType: chance
rules:
- condition: contains
  property: id
  value: zuganglichkeit-247

docs/anwendungsfälle/erkennenvonlernschwierigkeiten.md
[chance=zuganglichkeit-247]
condition: or
entityType: chance
rules:
- condition: contains
  property: id
  value: zuganglichkeit-247

docs/anwendungsfälle/textuebersetzung.md
[chance=zuganglichkeit-247]
condition: or
entityType: chance
rules:
- condition: contains
  property: id
  value: zuganglichkeit-247



In [ ]:
import re
import yaml

for tool in tools:
    with open(tool, 'r') as f:
        content = f.read()
    content = content.split("---")
    if len(content) > 1:

        metadata = content[1]
        metadata = yaml.load(metadata, Loader=yaml.FullLoader)
        if metadata.get('tags'):
            if not metadata.get("id"):
                print("----")
                print(tool)
                print("no id")
                print(metadata['tags'])
                print(tool.split('/')[-1].split('.')[0])
                metadata['id'] = tool.split('/')[-1].split('.')[0]
                print(yaml.dump(metadata))
                content[1] = yaml.dump(metadata)
                content = "---\n".join(content)
                print(content)
                with open(tool, 'w') as f:
                    f.write(content)

